This notebook aim to train all models on all datasets

In [ ]:
import os
os.chdir(os.path.dirname(os.getcwd()))

from load_matrix import load_df
from model_trainer import MLModel

Experimenting with different constants and parameters for the models

In [2]:
MLModel.set_global_variable('DEFAULT_SPLIT_RATIO', 0.3)

## Exploration in MLP

In [1]:
from sklearn.neural_network import MLPClassifier

help(MLPClassifier)

Help on class MLPClassifier in module sklearn.neural_network._multilayer_perceptron:

class MLPClassifier(sklearn.base.ClassifierMixin, BaseMultilayerPerceptron)
 |  MLPClassifier(
 |      hidden_layer_sizes=(100,),
 |      activation='relu',
 |      *,
 |      solver='adam',
 |      alpha=0.0001,
 |      batch_size='auto',
 |      learning_rate='constant',
 |      learning_rate_init=0.001,
 |      power_t=0.5,
 |      max_iter=200,
 |      shuffle=True,
 |      random_state=None,
 |      tol=0.0001,
 |      verbose=False,
 |      warm_start=False,
 |      momentum=0.9,
 |      nesterovs_momentum=True,
 |      early_stopping=False,
 |      validation_fraction=0.1,
 |      beta_1=0.9,
 |      beta_2=0.999,
 |      epsilon=1e-08,
 |      n_iter_no_change=10,
 |      max_fun=15000
 |  )
 |
 |  Multi-layer Perceptron classifier.
 |
 |  This model optimizes the log-loss function using LBFGS or stochastic
 |  gradient descent.
 |
 |  .. versionadded:: 0.18
 |
 |  Parameters
 |  ----------
 |

In [ ]:
df=load_df('gene_expression')
y=df['disease_status'].values
X=df.drop(columns=['disease_status']).values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [4]:
clf=MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)

In [9]:
clf.fit(X_train, y_train)

,hidden_layer_sizes,"(100,)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,200
,shuffle,True
,random_state,None


In [10]:
# -- after fitting
from sklearn.metrics import accuracy_score
y_pred=clf.predict(X_test)
y_test,y_pred, accuracy_score(y_test, y_pred)

(array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 0.17647058823529413)

In [98]:
y=df['disease_status'].values
X=df.drop(columns=['disease_status']).values
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [99]:
# -- pytorch model
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
input_size = X_train.shape[1]
hidden_size = 100
output_size = 1
model = SimpleMLP(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)

In [100]:
# -- prepare data loaders
train_dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [101]:
clf=SimpleMLP(input_size, hidden_size, output_size)
optimizer = optim.Adam(clf.parameters(), lr=0.001)
num_epochs = 100

save_loss=[]
save_accuracy=[]
for epoch in range(num_epochs):
    accuracy=0
    for i, (x_, y_) in enumerate(train_loader):
        y_pred=clf(x_.float()).reshape(-1)
        loss=criterion(y_pred, y_.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        accuracy+=accuracy_score(y_.float(), (y_pred > 0.5).float())

    if (epoch+1) % 5 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy/len(train_loader):.4f}')
    save_loss.append(loss.item())
    # save_accuracy.append(accuracy_score(y_train, y_pred.argmax(dim=1)))

Epoch [5/100], Loss: 27.7778, Accuracy: 0.7821
Epoch [10/100], Loss: 11.1111, Accuracy: 0.8003
Epoch [15/100], Loss: 16.6667, Accuracy: 0.7943
Epoch [20/100], Loss: 11.1111, Accuracy: 0.8003
Epoch [25/100], Loss: 22.2222, Accuracy: 0.7882
Epoch [30/100], Loss: 16.6667, Accuracy: 0.7943
Epoch [35/100], Loss: 16.6667, Accuracy: 0.7943
Epoch [40/100], Loss: 27.7778, Accuracy: 0.7821
Epoch [45/100], Loss: 11.1111, Accuracy: 0.8003
Epoch [50/100], Loss: 11.1111, Accuracy: 0.8003
Epoch [55/100], Loss: 11.1111, Accuracy: 0.8003
Epoch [60/100], Loss: 27.7778, Accuracy: 0.7821
Epoch [65/100], Loss: 27.7778, Accuracy: 0.7821
Epoch [70/100], Loss: 22.2222, Accuracy: 0.7882
Epoch [75/100], Loss: 11.1111, Accuracy: 0.8003
Epoch [80/100], Loss: 27.7778, Accuracy: 0.7821
Epoch [85/100], Loss: 22.2222, Accuracy: 0.7882
Epoch [90/100], Loss: 11.1111, Accuracy: 0.8003
Epoch [95/100], Loss: 11.1111, Accuracy: 0.8003
Epoch [100/100], Loss: 5.5556, Accuracy: 0.8064


In [102]:
y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0])

In [103]:
# -- test
y_preds=[]
y_true=y_test.tolist()
for i, (x, y) in enumerate(test_loader):
    y_pred=clf(x.float()).reshape(-1)
    
print(y_pred)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       grad_fn=<ViewBackward0>)


Tried pytorch and sklearn implementations and different hyperparameters but looks like the model always tend to predct the label 1 when tested on the test set